In [1]:
#To frame the data
import pandas as pd
import numpy as np

#to scrape the data
import requests
import time

#to read the data as html, text
from bs4 import BeautifulSoup

#to characterize the text data
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
cv = CountVectorizer

In [3]:
URL = "https://www.reddit.com"## YOUR CODE HERE
response = requests.get(URL, headers = {'User-agent': 'MrMagic'})
html = response.text
soup = BeautifulSoup(html, 'lxml')

In [5]:
def list_subreddits(iterations= 2, user_output= 'A'):
    
    #re-instantiate iterations
    itera = iterations
    
    user_iterations = input('Iterations/pages (default = 2): ')
    if user_iterations != '':
        itera = int(user_iterations)
        
    
    #defaults to the full dataframe, any other argument gives a test output
    user_output_choice = user_output
    choice = input('"\n"Type A for the full dataframe (default).\
                   "\n"Type B for the data withouth central tendencies.\
                   "\n"Type C for the central tendencies.\
                   "\n"Type D for the subbreddit linksoup.')

    if choice != '':
        user_output_choice = choice
    
    
    #go to URL, turn the page to text, get html 
    URL = "https://www.reddit.com"## YOUR CODE HERE
    response = requests.get(URL, headers = {'User-agent': 'MrMagic'})
    html = response.text
    soup = BeautifulSoup(html, 'lxml')
    
    #create an empty dictionary
    dicti = {}
    
    #create empty lists for the title, time, and subreddit category, and comment
    list_title = []
    list_time = []
    list_subs = []
    list_comments = []
    
    
    #comments will have to be stripped of their string values, leaving the comment counts
    list_comment_counts = []
    
    
    #dump the link soup here
    list_linksoup = []
    
    #comment rate is the comment count/hours
    list_cRate = []
    
    #gather links
    list_p = []
    list_a = []

    while itera > 0:
        
        try:
        
            #identify the tags containing the features of interest
            title_tags = soup.find_all('a', {'data-event-action' : "title"})
            time_tags = soup.find_all('time',  {'class':'live-timestamp'})
            sub_tags = soup.find_all('a', {'class':'subreddit hover may-blank'})
            sub_comments = soup.find_all('a', {'data-event-action':'comments'})
            link = soup.find_all('span', {'class': 'next-button'})
            p_set = soup.find_all('p', {'class': 'title'})
            
            #append various feature lists
            for tag in title_tags:
                list_title.append(tag.text)
            for tag in time_tags:
                list_time.append(tag.text)
            for tag in sub_tags:
                list_subs.append(tag.text)
            for tag in sub_comments:
                list_comments.append(tag.text)
            for i in range(len(p_set)):
                list_p.append(soup.find_all('p', {'class': 'title'})[i])
                list_a.append(list_p[i]('a')[0]['href'])
            
    


            #the next button link URL based on the comment count of 25 per page, assigned to variable 'link'
            link_tag = soup.find_all('span', {'class', 'next-button'})
            link = link_tag[0]('a')[0]['href']
            response = requests.get(link, headers = {'User-agent': 'MrMagic'})
            html = response.text
            soup = BeautifulSoup(html, 'lxml')

            #add a sleep command
            time.sleep(1) 

            itera -= 1
        except:
            pass
    
   
    for i in range(len(list_a)):
        if list_a[i][:2] == '/r':
            list_a[i] = 'https://www.reddit.com' + list_a[i]    
    for i in list_comments:
        list_comment_counts.append(i[:-9])
    for lst in list_time:
        list_cRate.append(lst)
    for i in range(len(list_cRate)):
        list_cRate[i] = int(list_comment_counts[i])/float(list_cRate[i][:-9].strip()) #ugly float
    for i in range(len(list_a)):
        url = list_a[i]
        response = requests.get(URL, headers = {'User-agent': 'MrGumbo'})
        html = response.text
        soup = BeautifulSoup(html, 'lxml')
        list_linksoup.append(soup)
    
    
    #cleaning up the comment rate output
    list_cRate = [ '%.1f' % i for i in list_cRate]                  
    list_cRate = [float(i) for i in list_cRate] 
    
    
   ################################################################## 

     # comment count central tendancies
    list_comment_counts = [float(i) for i in list_comment_counts]
    
    list_comment_median_threshold = []
    for i in range(len(list_comment_counts)):
        if list_comment_counts[i] >= np.median(list_comment_counts):
            list_comment_median_threshold.append(1)
        else:
            list_comment_median_threshold.append(0)
    
    list_comment_mean_threshold = []
    for i in range(len(list_comment_counts)):
        if list_comment_counts[i] >= np.mean(list_comment_counts):
            list_comment_mean_threshold.append(1)
        else:
            list_comment_mean_threshold.append(0)
    
    # comment rate central tendancies
    list_cRate_mean_threshold = []
    for i in range(len(list_cRate)):
        if list_cRate[i] >= np.mean(list_cRate):
            list_cRate_mean_threshold.append(1)
        else:
            list_cRate_mean_threshold.append(0)
            
    list_cRate_median_threshold = []
    for i in range(len(list_cRate)):
        if list_cRate[i] >= np.median(list_cRate):
            list_cRate_median_threshold.append(1)
        else:
            list_cRate_median_threshold.append(0)     
            
    ##################################################################  
    
    
    
    
    
    #building the dictionary
    dicti['Above_comment_mean'] = list_comment_mean_threshold
    dicti['Above_comment_median'] = list_comment_median_threshold
    dicti['Above_cRate_mean'] = list_cRate_mean_threshold
    dicti['Above_cRate_median'] = list_comment_median_threshold
    dicti['subreddit_linksoup'] = list_linksoup
    dicti['title'] = list_title
    dicti['time'] = list_time
    dicti['subreddit'] = list_subs
    dicti['comment_counts'] = list_comment_counts
    dicti['sub_link'] = list_a
    dicti['comment_per_hour'] = list_cRate
    
    #convert the dictionary to a dataframe, sorted by the comment rate in descending order
    pg_df = pd.DataFrame(dicti)
    pg_df = pg_df.sort_values(['comment_per_hour'], ascending = False)  #sort by comment rate
                               
    thresh = pd.DataFrame({'comment_mean': list_comment_median_threshold,
                           'comment_median': list_comment_median_threshold,
                           'cRate_mean': list_cRate_mean_threshold,
                           'cRate_median': list_cRate_median_threshold})
    #return pg_df[['title', 'subreddit', 'sub_link', 'comment_counts','comment_per_hour', 'time', ]
    

    df1 = pg_df[['title', 'subreddit', 'sub_link', 'comment_counts','comment_per_hour', 'time', ]]
    df2 = pg_df[['title', 'Above_comment_mean', 'Above_comment_median', 'Above_cRate_mean', 'Above_cRate_median']]
    df3 = pg_df[['title', 'subreddit_linksoup']]
    
    if user_output_choice.lower() == 'a':
        return pg_df
    elif user_output_choice.lower() == 'b':
        return df1
    elif user_output_choice.lower() == 'c':
        return df2
    elif user_output_choice.lower() == 'd':
        return df3
    else:
        None
            
                               
                               
                               
                               
                               
   

In [6]:
# 1 second sleep timer
Reddit_analyzer = list_subreddits()

Iterations/pages (default = 2): 2
"
"Type A for the full dataframe (default).                   "
"Type B for the data withouth central tendencies.                   "
"Type C for the central tendencies.                   "
"Type D for the subbreddit linksoup.


In [10]:
Reddit_analyzer.head(2)

,Above_cRate_mean,Above_cRate_median,Above_comment_mean,Above_comment_median,comment_counts,comment_per_hour,sub_link,subreddit,subreddit_linksoup,time,title
12,1,1,1,1,2972.0,495.3,https://i.imgur.com/mThH44E.gifv,r/gaming,"<!DOCTYPE html> <html lang=""en"" xml:lang=""en"" ...",6 hours ago,This is how i see my retirement
32,1,1,0,1,431.0,431.0,https://www.reddit.com/r/pics/comments/80dbt6/...,r/nottheonion,"<!DOCTYPE html> <html lang=""en"" xml:lang=""en"" ...",1 hour ago,President Trump: I would have run into school ...


In [9]:
Reddit_analyzer.shape

(50, 11)

In [ ]:
cv(Reddit_analyzer['text_content'], stop_words="english")

#### Begin Greg code below

In [ ]:
# Count vectorize the title field
cv = CountVectorizer(stop_words='english')
cv_vals = cv.fit_transform(Reddit_analyzer['title'])

In [59]:
# create X df from title (count vectorized) and subreddit (get dummied)
X = pd.concat([pd.get_dummies(Reddit_analyzer['subreddit']).set_index(np.arange(pd.get_dummies(Reddit_analyzer['subreddit']).shape[0]), inplace=True)
           , pd.DataFrame(cv_vals.todense(), columns=cv.get_feature_names())]
          , ignore_index=False
          , axis=1
)
# create y from CRate and comment median fields (this is a 50x4 y target, not a single class)
y = Reddit_analyzer.iloc[:,:4]

In [65]:
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

In [62]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [66]:
# multi output classifier allows us to do a multi-column y target with a normal classifier like logistic regression
moc = MultiOutputClassifier(LogisticRegression())

In [67]:
moc.fit(X_train, y_train)

MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1)

In [69]:
# testing accuracy of the model
moc.score(X_test, y_test)

0.46666666666666667

In [84]:
# list the column headers of the predicted y target
y_test.columns.tolist()

['Above_cRate_mean',
 'Above_cRate_median',
 'Above_comment_mean',
 'Above_comment_median']

In [78]:
# probability class labels of 'Above_cRate_mean', 'Above_cRate_median', 'Above_comment_mean', 'Above_comment_median'
# i.e. [ 0.72292577  0.27707423] is a 72% probability of Above_cRate_mean being 1 and 27% probability of being 0
for i, j, k, l in zip(moc.predict_proba(X_test)[0], 
                moc.predict_proba(X_test)[1],
                moc.predict_proba(X_test)[2],
                moc.predict_proba(X_test)[3]):
    print(i,j, k, l)

[ 0.72292577  0.27707423] [ 0.50189839  0.49810161] [ 0.66034715  0.33965285] [ 0.50189839  0.49810161]
[ 0.72292577  0.27707423] [ 0.50189839  0.49810161] [ 0.66034715  0.33965285] [ 0.50189839  0.49810161]
[ 0.58852083  0.41147917] [ 0.39543286  0.60456714] [ 0.52939052  0.47060948] [ 0.39543286  0.60456714]
[ 0.72292577  0.27707423] [ 0.50189839  0.49810161] [ 0.66034715  0.33965285] [ 0.50189839  0.49810161]
[ 0.72292577  0.27707423] [ 0.50189839  0.49810161] [ 0.66034715  0.33965285] [ 0.50189839  0.49810161]
[ 0.72292577  0.27707423] [ 0.50189839  0.49810161] [ 0.66034715  0.33965285] [ 0.50189839  0.49810161]
[ 0.72292577  0.27707423] [ 0.50189839  0.49810161] [ 0.66034715  0.33965285] [ 0.50189839  0.49810161]
[ 0.72292577  0.27707423] [ 0.50189839  0.49810161] [ 0.66034715  0.33965285] [ 0.50189839  0.49810161]
[ 0.72292577  0.27707423] [ 0.50189839  0.49810161] [ 0.66034715  0.33965285] [ 0.50189839  0.49810161]
[ 0.74471064  0.25528936] [ 0.46102994  0.53897006] [ 0.61341479

# EVERYTHING BELOW THIS CELL IS NOTES ONLY

## Stay above this code for practice/scratch... the goal is to iterate beyone 20 pages

In [ ]:
def list_subreddits(iterations= 1):
    
    #go to URL, turn the page to text, get html 
    URL = "https://www.reddit.com"## YOUR CODE HERE
    response = requests.get(URL, headers = {'User-agent': 'MrMagic'})
    html = response.text
    soup = BeautifulSoup(html, 'lxml')
    
    dicti = {}
    list_title = []
    list_time = []
    list_subs = []
    list_comments = []
    list_comment_counts = []
    list_p = []
    list_a = []sleep_val = iteration - 4
    list_linksoup = []
    list_cRate = []
    
    itera = iterations
    
    if iterations > 19:
        iterad = iteration / 19
        iterar = iteration % 19
        
    n_dataframes = [iterad + 1 if iterar > 0 else iterad]
    lst_dfs = []
    for i in range(n_dataframes):
        lst_dfs.append('df_'+ str(i))
    
    while itera > 0:
        
        try:
        
            #identify the tags containing the features of interest
            title_tags = soup.find_all('a', {'data-event-action' : "title"})
            time_tags = soup.find_all('time',  {'class':'live-timestamp'})
            sub_tags = soup.find_all('a', {'class':'subreddit hover may-blank'})
            sub_comments = soup.find_all('a', {'data-event-action':'comments'})
            link = soup.find_all('span', {'class': 'next-button'})
            p_set = soup.find_all('p', {'class': 'title'})

            #append various feature lists
            for tag in title_tags:
                list_title.append(tag.text)
            for tag in time_tags:
                list_time.append(tag.text)
            for tag in sub_tags:
                list_subs.append(tag.text)
            for tag in sub_comments:
                list_comments.append(tag.text)
            for i in range(len(p_set)):
                list_p.append(soup.find_all('p', {'class': 'title'})[i])
                list_a.append(list_p[i]('a')[0]['href'])


            #the next button link URL based on the comment count of 25 per page, assigned to variable 'link'
            link_tag = soup.find_all('span', {'class', 'next-button'})
            link = link_tag[0]('a')[0]['href']
            response = requests.get(link, headers = {'User-agent': 'MrMagic'})
            html = response.text
            soup = BeautifulSoup(html, 'lxml')

            #add a sleep command
            time.sleep(1) #defined as a function of number of iterations

            itera -= 1
        except:
            pass
    
    for i in range(len(list_a)):
        if list_a[i][:2] == '/r':
            list_a[i] = 'https://www.reddit.com' + list_a[i]    
    for i in list_comments:
        list_comment_counts.append(i[:-9])
    for lst in list_time:
        list_cRate.append(lst)
    for i in range(len(list_cRate)):
        list_cRate[i] = float(list_comment_counts[i])/int(list_cRate[i][:-9]) #ugly float
    
    list_cRate = [ '%.1f' % i for i in list_cRate]                  #tidy float
    list_cRate = [float(i) for i in list_cRate]                     #had to be redundant for some reason?
    dicti['title'] = list_title
    dicti['time'] = list_time
    dicti['subreddit'] = list_subs
    dicti['comment_counts'] = list_comment_counts
    dicti['sub_link'] = list_a
    dicti['comment_per_hour'] = list_cRate
    
    #convert the dictionary to a dataframe, sorted by the comment rate in descending order
    pg_df = pd.DataFrame(dicti)
    pg_df = pg_df.sort_values(['comment_per_hour'], ascending = False)  #sort by comment rate
    #pg_df = pg_df[pg_df.time != '1 hour ago']
    
    
    #return pg_df[['title', 'subreddit', 'sub_link', 'comment_counts', 'time']]
    return pg_df[['title', 'subreddit', 'sub_link', 'comment_counts','comment_per_hour', 'time', ]]


In [ ]:
length = len(list_subreddits(5))   
length

In [ ]:
nineteen_iterations = list_subreddits(19)

In [ ]:
ten_iterations = list_subreddits(10)

In [ ]:
len(ten_iterations)
ten_iterations
#next, try sort_values by time then comment_count,

In [ ]:
list_subreddits()['comment_per_hour'] = [float(i) for i in list_subreddits()['comment_per_hour']]
tst
tst = [float(i) for i in tst]
tst
tst.sort()
tst

In [ ]:
#lengths = [len(list_subreddits(i)) for i in range(8,12,2)].... does not work
    #time.sleep(3)
    
#time.sleep(3)    
    #length = [len(list_subreddits(i)) for i in range(4,8,2)]... does work > [100, 150]
    #length = len(list_subreddits(7))... does work > 175
    #length = len(list_subreddits(8))... does not work > ValueError: invalid literal for int() with base 10: ''

#time.sleep(4)
    #length = len(list_subreddits(7))... does work > 175
    #length = len(list_subreddits(8))... does work > 200
    #length = len(list_subreddits(10))... does not work
    #length = len(list_subreddits(9)) ... does not work 
    #length = len(list_subreddits(8))... does work > 200
    
#time.sleep(5)
    #length = len(list_subreddits(8))... does work > 200
    #length = len(list_subreddits(9))... does work > 225
    #length = len(list_subreddits(10))... does not work

#time.sleep(6)
    #length = len(list_subreddits(8))... does work
    #length = len(list_subreddits(10))...

#wo
    


#### Figure out how to list subreddit links